# Segmenting and Clustering Neighborhoods in Toronto

This is the notebook assciated with the assignment "Segment and Clustering Neighborhoods in Toronto". It is structured in three parts:

1. <a href="#item1">Download Dataset and Build Dataframe for Toronto Neighborhoods</a>    
2. <a href="#item2">Enrich Dataset with Geolocation Information</a> 
3. <a href="#item3">Search for Venues, Segment and Cluster Neighborhoods</a> 


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191113192744-0002
KERNEL_ID = 361f1b10-c27b-4907-8c60-3bb08c9b81f6
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<a id='item1'></a>

## 1. Download Dataset and Build Dataframe for Toronto Neigborhoods 

### 1.1. Load data from Wikipedia

Now we will load brute data concerning the Neighbourhood in Toronto. We will store this in a Datafarme called all_nb

In [2]:
#import the table using the pandas read_html method
#take the first returned object as teh brute Neighbourghood 
pd_list = pd.read_html ("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

all_nb = pd_list[0]
all_nb.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Check the shape of the initial Dataframe. Apparently, we have 233 lines

In [3]:
all_nb.shape

(288, 3)

### 1.2. Clean data

Drop the lines whith 'Not assigned' borough. There are 211 lines remaining

In [4]:
all_nb.drop(all_nb[all_nb['Borough']=="Not assigned"].index, axis=0, inplace=True)
all_nb.shape

(211, 3)

Set Neighbourhood to Borough for the lines where Neighbourhood is 'Not Assigned'

In [5]:
all_nb.loc[all_nb['Neighbourhood']=="Not assigned", 'Neighbourhood'] = all_nb.loc[all_nb['Neighbourhood']=="Not assigned"]['Borough']

Sort the remaining rows ans reset the index. Then, let's take a look at how many neighbourhoods we have per borough.

In [6]:
all_nb.sort_values(by=['Postcode', 'Borough', 'Neighbourhood'], inplace=True)
all_nb.reset_index(inplace=True, drop=True)

all_nb.groupby(['Postcode', 'Borough']).count()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,2
M1C,Scarborough,3
M1E,Scarborough,3
M1G,Scarborough,1
M1H,Scarborough,1
M1J,Scarborough,1
M1K,Scarborough,3
M1L,Scarborough,3
M1M,Scarborough,3


### 1.3. Generate the Final Neighbourhood Dataframe

Concatenate the names of the neighbourhoods which belong to the the same borough. 

The final dataframe is called **toronto_nbhood**

In [7]:
# define the dataframe columns
column_names = ['Postcode', 'Borough', 'Neighbourhood'] 

# instantiate the dataframe
toronto_nbhood = pd.DataFrame(columns=column_names)
toronto_nbhood = toronto_nbhood.append(all_nb.loc[0])

j = 0
for i in range(1, all_nb.shape[0]):
    Postcode = all_nb.iloc[i]['Postcode']
    Borough = all_nb.iloc[i]['Borough']
    Neighbourhood = all_nb.iloc[i]['Neighbourhood']
    if toronto_nbhood.iloc[j]['Postcode'] == Postcode and toronto_nbhood.iloc[j]['Borough'] == Borough:
        toronto_nbhood.iloc[j]['Neighbourhood'] = toronto_nbhood.iloc[j]['Neighbourhood'] + ', ' + Neighbourhood
    else:
        toronto_nbhood = toronto_nbhood.append([{'Postcode':Postcode, 'Borough':Borough, 'Neighbourhood':Neighbourhood}], ignore_index=True)
        j = j + 1
   
toronto_nbhood

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


How many lines do we have now? We should find 103.

In [8]:
toronto_nbhood.shape

(103, 3)

<hr>

<a id='item2'></a>

## 2. Enrich Dataset with Geospatial Information

I've tryed to use geocoder library unsucessfully.

I will load the prepared csv file of geospatial coordinates

In [9]:
file_name='https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
df_geo=pd.read_csv(file_name)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Generate **toronto_nbhood_geo** dataframe by enriching **toronto_nbhood** with the geospatial coordinates. Use panda join function 

In [10]:
toronto_nbhood_geo=toronto_nbhood.join(df_geo.set_index('Postal Code'), on='Postcode')

In [11]:
toronto_nbhood_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


The enriched dataframe should have 103 lines and 5 columns.

In [12]:
toronto_nbhood_geo.shape

(103, 5)

<hr>

<a id='item3'></a>

## 3. Search for Venues, Segment and Cluster Neighborhoods


First, let's find the geographical coordinates of Toronto, using the geopy library.

In [13]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### 3.1.Create a map of Toronto with neighbourhoods superimposed on top.

In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_nbhood_geo['Latitude'], toronto_nbhood_geo['Longitude'], toronto_nbhood_geo['Borough'], toronto_nbhood_geo['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Keep only boroughs in Toronto and save them into a specific **toronto_data** dataframe

In [15]:
toronto_data = toronto_nbhood_geo[toronto_nbhood_geo['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Let's zoom on central Toronto and show only the boroughs containing the name 'Toronto' 

In [18]:
# create map of Toronot using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.2. Explore Neighborhoods in Toronto; Load Venues

In [17]:
# @hidden_cell
CLIENT_ID = 'WRGTBGWVIWD1S0LLK0GRGEARWM2GBYEQ1QOP0PSIPNBJHOUT' # your Foursquare ID
CLIENT_SECRET = 'VALOPUUYGGEAIWKNIW5WVQRIIGYRO2EZJ0LOTKT1JDXXECKF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Let's create a function to search for all the neighborhoods in Toronto. Use a large radius (800m) for each search, in order to get many venues, especially for neighborhoods far from center

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=800, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                 
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        #print(url)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        print(name, ": ", len(results), " venues")
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now run the above function on each neighborhood and create a new dataframe called **toronto_venues_with_duplicates**.

In [20]:
# fill in all venues

toronto_venues_with_duplicates = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches :  46  venues
Riverdale, The Danforth West :  100  venues
India Bazaar, The Beaches West :  53  venues
Studio District :  98  venues
Lawrence Park :  7  venues
Davisville North :  52  venues
North Toronto West :  33  venues
Davisville :  79  venues
Moore Park, Summerhill East :  11  venues
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West :  64  venues
Rosedale :  8  venues
Cabbagetown, St. James Town :  72  venues
Church and Wellesley :  100  venues
Harbourfront, Regent Park :  86  venues
Garden District, Ryerson :  100  venues
St. James Town :  100  venues
Berczy Park :  100  venues
Central Bay Street :  100  venues
Adelaide, King, Richmond :  100  venues
Harbourfront East, Toronto Islands, Union Station :  100  venues
Design Exchange, Toronto Dominion Centre :  100  venues
Commerce Court, Victoria Hotel :  100  venues
Roselawn :  5  venues
Forest Hill North, Forest Hill West :  16  venues
North Midtown, The Annex, Yorkville :  78  venues
Harbord, Universi

We have 2691 venues, but there are a lot of **duplicates** inside.

In [21]:
print(toronto_venues_with_duplicates.shape)
toronto_venues_with_duplicates.sort_values(by=['Venue', 'Venue Latitude', 'Venue Longitude']).head(20)

(2691, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
295,Studio District,43.659526,-79.340923,1-800-GOT-JUNK? Toronto,43.654972,-79.347811,Home Service
619,"Cabbagetown, St. James Town",43.667967,-79.367675,166 Carlton Convenience,43.663124,-79.373711,Grocery Store
2452,"High Park, The Junction South",43.661608,-79.464763,3030 Dundas,43.665574,-79.470376,Bar
840,"Garden District, Ryerson",43.657162,-79.378937,306 Yonge Street - Jordan Store,43.656495,-79.381015,Sporting Goods Shop
1193,Central Bay Street,43.657952,-79.387383,306 Yonge Street - Jordan Store,43.656495,-79.381015,Sporting Goods Shop
1360,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,360 Restaurant,43.642537,-79.387042,Wine Bar
1884,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,4 Life Natural Foods,43.655275,-79.402374,Farmers Market
644,Church and Wellesley,43.665860,-79.383160,401 Games,43.663623,-79.384037,Hobby Shop
1194,Central Bay Street,43.657952,-79.387383,401 Games,43.663623,-79.384037,Hobby Shop
1663,"North Midtown, The Annex, Yorkville",43.672710,-79.405678,5 Elements Espresso,43.673695,-79.396037,Café


### 3.3. Suppress duplicates from Venues

Calculate distance to the neighborhood for each Venue

In [22]:
toronto_venues_with_duplicates['Distance']=np.sqrt((toronto_venues_with_duplicates['Neighborhood Latitude']-toronto_venues_with_duplicates['Venue Latitude'])**2\
                                   +(toronto_venues_with_duplicates['Neighborhood Longitude']-toronto_venues_with_duplicates['Venue Longitude'])**2)

In [23]:
toronto_venues_with_duplicates.sort_values(by=['Venue', 'Venue Latitude', 'Venue Longitude', 'Distance'], inplace=True)
toronto_venues_with_duplicates.reset_index(inplace=True, drop=True)
toronto_venues_with_duplicates.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance
0,Studio District,43.659526,-79.340923,1-800-GOT-JUNK? Toronto,43.654972,-79.347811,Home Service,0.008257
1,"Cabbagetown, St. James Town",43.667967,-79.367675,166 Carlton Convenience,43.663124,-79.373711,Grocery Store,0.007738
2,"High Park, The Junction South",43.661608,-79.464763,3030 Dundas,43.665574,-79.470376,Bar,0.006872
3,"Garden District, Ryerson",43.657162,-79.378937,306 Yonge Street - Jordan Store,43.656495,-79.381015,Sporting Goods Shop,0.002183
4,Central Bay Street,43.657952,-79.387383,306 Yonge Street - Jordan Store,43.656495,-79.381015,Sporting Goods Shop,0.006532
5,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,360 Restaurant,43.642537,-79.387042,Wine Bar,0.005563
6,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,4 Life Natural Foods,43.655275,-79.402374,Farmers Market,0.003112
7,Church and Wellesley,43.665860,-79.383160,401 Games,43.663623,-79.384037,Hobby Shop,0.002403
8,Central Bay Street,43.657952,-79.387383,401 Games,43.663623,-79.384037,Hobby Shop,0.006584
9,"North Midtown, The Annex, Yorkville",43.672710,-79.405678,5 Elements Espresso,43.673695,-79.396037,Café,0.009691


Eliminate duplicates. Attach each unique venue to its closest Neighbourhood

In [24]:
# define the dataframe columns
column_names = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category', 'Distance'] 

# instantiate the dataframe
toronto_venues = pd.DataFrame(columns=column_names)
toronto_venues = toronto_venues.append(toronto_venues_with_duplicates.loc[0])

j = 0
dup = 0

ven_unique = (toronto_venues.iloc[j]['Venue'], '%.6f' % toronto_venues.iloc[j]['Venue Latitude'], \
              '%.6f' % toronto_venues.iloc[j]['Venue Longitude'], toronto_venues.iloc[j]['Venue Category'])


for i in range(1, toronto_venues_with_duplicates.shape[0]):
    ven_dup = (toronto_venues_with_duplicates.iloc[i]['Venue'], '%.6f' % toronto_venues_with_duplicates.iloc[i]['Venue Latitude'], \
               '%.6f' % toronto_venues_with_duplicates.iloc[i]['Venue Longitude'], toronto_venues_with_duplicates.iloc[i]['Venue Category'])
    if ven_unique != ven_dup:
        toronto_venues = toronto_venues.append(toronto_venues_with_duplicates.iloc[i], ignore_index=True)
        j = j + 1
        ven_unique = (toronto_venues.iloc[j]['Venue'], '%.6f' % toronto_venues.iloc[j]['Venue Latitude'], \
                      '%.6f' % toronto_venues.iloc[j]['Venue Longitude'], toronto_venues.iloc[j]['Venue Category'])
    else:
        dup = dup + 1

print (dup)
toronto_venues.head(10)


733


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance
0,Studio District,43.659526,-79.340923,1-800-GOT-JUNK? Toronto,43.654972,-79.347811,Home Service,0.008257
1,"Cabbagetown, St. James Town",43.667967,-79.367675,166 Carlton Convenience,43.663124,-79.373711,Grocery Store,0.007738
2,"High Park, The Junction South",43.661608,-79.464763,3030 Dundas,43.665574,-79.470376,Bar,0.006872
3,"Garden District, Ryerson",43.657162,-79.378937,306 Yonge Street - Jordan Store,43.656495,-79.381015,Sporting Goods Shop,0.002183
4,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,360 Restaurant,43.642537,-79.387042,Wine Bar,0.005563
5,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,4 Life Natural Foods,43.655275,-79.402374,Farmers Market,0.003112
6,Church and Wellesley,43.665860,-79.383160,401 Games,43.663623,-79.384037,Hobby Shop,0.002403
7,"North Midtown, The Annex, Yorkville",43.672710,-79.405678,5 Elements Espresso,43.673695,-79.396037,Café,0.009691
8,"Riverdale, The Danforth West",43.679557,-79.352188,7 Numbers,43.677062,-79.353934,Italian Restaurant,0.003046
9,"Forest Hill North, Forest Hill West",43.696948,-79.411307,7 Numbers,43.703630,-79.413724,Italian Restaurant,0.007107


How many venues per neighborhood do we have now?

In [25]:
toronto_venues[['Neighborhood', 'Venue']].groupby(['Neighborhood']).count()

,Venue
Neighborhood,
"Adelaide, King, Richmond",54
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",25
Berczy Park,16
"Brockton, Exhibition Place, Parkdale Village",96
Business Reply Mail Processing Centre 969 Eastern,44
"Cabbagetown, St. James Town",72
Central Bay Street,48
"Chinatown, Grange Park, Kensington Market",99
Christie,52


Finally, we have 1958 venues to analyze

In [26]:
toronto_venues.shape

(1958, 8)

### 3.4. Analyze Each Neighborhood

#### Let's find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


Let's set up the number of venues as columns

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood Name'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(10)

,Neighborhood Name,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Studio District,0,0,0,0,0,

In [29]:
toronto_onehot.shape

(1958, 277)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood Name').mean().reset_index()
toronto_grouped

,Neighborhood Name,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond"

#### Let's confirm the new size

In [31]:
toronto_grouped.shape

(38, 277)

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood Name'] = toronto_grouped['Neighborhood Name']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Sushi Restaurant,Pizza Place,Asian Restaurant,Theater,Thai Restaurant,Hotel,Café,Smoke Shop
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",Harbor / Marina,Airport Service,Boat or Ferry,Sculpture Garden,Coffee Shop,Airport Lounge,Boutique,Tunnel,Bar,Dog Run
2,Berczy Park,Coffee Shop,Pizza Place,Hotel,Bistro,Molecular Gastronomy Restaurant,Liquor Store,Fish Market,Beach,Restaurant,Creperie
3,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Bakery,Furniture / Home Store,Restaurant,Arts & Crafts Store,Bar,Café,Italian Restaurant,Music Venue,Lounge
4,Business Reply Mail Processing Centre 969 Eastern,Fast Food Restaurant,Pizza Place,Bakery,Coffee Shop,Pub,Brewery,Bar,Light Rail Station,Harbor / Marina,Grocery Store


### 3.5. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 7 clusters.

In [46]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, max_iter=1000, random_state=1).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
print (kmeans.labels_[0:40])

[3 6 5 5 3 5 0 3 5 3 0 3 3 0 0 5 3 0 3 0 5 0 3 3 0 5 1 3 0 3 5 2 4 0 0 5 3
 3]


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [47]:
# add clustering labels
neighborhoods_venues_sorted.drop('Cluster Labels', 1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood Name'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Bar,Breakfast Spot,Sandwich Place,Café,Japanese Restaurant,Grocery Store,Beach,Bookstore,Caribbean Restaurant
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,5,Greek Restaurant,Coffee Shop,Pub,Café,Ice Cream Shop,Fast Food Restaurant,Italian Restaurant,Park,Yoga Studio,Discount Store
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,Indian Restaurant,Grocery Store,Gym,Sandwich Place,Burger Joint,Donut Shop,Discount Store,Coffee Shop,Café,Pet Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Sandwich Place,Bar,Bakery,Coffee Shop,Diner,Brewery,Pizza Place,Arts & Crafts Store,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bookstore,Café,Coffee Shop,Gym / Fitness Center,Park,Restaurant,Bus Line,Yoga Studio,Dumpling Restaurant,Dog Run


Finally, let's visualize the resulting clusters

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.6. Try to describe the clusters

#### Cluster 0 - Urban Area

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,0,Bookstore,Café,Coffee Shop,Gym / Fitness Center,Park,Restaurant,Bus Line,Yoga Studio,Dumpling Restaurant,Dog Run
6,North Toronto West,0,Coffee Shop,Sporting Goods Shop,Italian Restaurant,Skating Rink,Café,Diner,Garden,Chinese Restaurant,Spa,Salon / Barbershop
9,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Pub,Gym,Bagel Shop,Sandwich Place,Restaurant,Café,Thai Restaurant
13,"Harbourfront, Regent Park",0,Coffee Shop,Park,Restaurant,Pub,Bakery,Theater,Italian Restaurant,Performing Arts Venue,Thai Restaurant,Café
15,St. James Town,0,Coffee Shop,Italian Restaurant,Breakfast Spot,Gastropub,Restaurant,Café,BBQ Joint,Hotel,Japanese Restaurant,Cosmetics Shop
17,Central Bay Street,0,Coffee Shop,Ice Cream Shop,Park,Japanese Restaurant,Chinese Restaurant,Bubble Tea Shop,Café,Art Gallery,Italian Restaurant,Tea Room
20,"Design Exchange, Toronto Dominion Centre",0,Coffee Shop,Restaurant,Italian Restaurant,Train Station,Café,Bakery,Spa,Fried Chicken Joint,Cocktail Bar,Hotel
21,"Commerce Court, Victoria Hotel",0,Coffee Shop,Café,Hotel,Seafood Restaurant,Bookstore,Japanese Restaurant,Italian Restaurant,Restaurant,Tea Room,Bakery
23,"Forest Hill North, Forest Hill West",0,Italian Restaurant,Park,Coffee Shop,Japanese Restaurant,Trail,Bakery,Bagel Shop,Sushi Restaurant,Gastropub,Café
25,"Harbord, University of Toronto",0,Café,Coffee Shop,Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Bakery,Beer Bar,Thai Restaurant,Pub,Japanese Restaurant


#### Cluster 1 - Moore Park - residential area

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park, Summerhill East",1,Park,Playground,Gym,Grocery Store,Thai Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Eastern European Restaurant,Design Studio


#### Cluster 2 - Rosedale - rich residential area

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,2,Park,Trail,Playground,Bank,Grocery Store,Candy Store,Yoga Studio,Doner Restaurant,Diner,Discount Store


#### Cluster 3 - Café, Bar, Pub, Restaurant Area

In [53]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,3,Pub,Bar,Breakfast Spot,Sandwich Place,Café,Japanese Restaurant,Grocery Store,Beach,Bookstore,Caribbean Restaurant
2,"India Bazaar, The Beaches West",3,Indian Restaurant,Grocery Store,Gym,Sandwich Place,Burger Joint,Donut Shop,Discount Store,Coffee Shop,Café,Pet Store
3,Studio District,3,Café,Sandwich Place,Bar,Bakery,Coffee Shop,Diner,Brewery,Pizza Place,Arts & Crafts Store,Italian Restaurant
5,Davisville North,3,Park,Food & Drink Shop,Italian Restaurant,Bar,Dessert Shop,Breakfast Spot,Sandwich Place,Deli / Bodega,Indian Restaurant,Sushi Restaurant
7,Davisville,3,Italian Restaurant,Sushi Restaurant,Coffee Shop,Café,Dessert Shop,Pizza Place,Fast Food Restaurant,Middle Eastern Restaurant,Ice Cream Shop,Indian Restaurant
12,Church and Wellesley,3,Coffee Shop,Japanese Restaurant,Café,Sushi Restaurant,Mediterranean Restaurant,Italian Restaurant,Bubble Tea Shop,Restaurant,Burger Joint,Pizza Place
14,"Garden District, Ryerson",3,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Coffee Shop,Diner,Middle Eastern Restaurant,Sporting Goods Shop,Italian Restaurant,Bookstore,Bakery
18,"Adelaide, King, Richmond",3,Coffee Shop,Steakhouse,Sushi Restaurant,Pizza Place,Asian Restaurant,Theater,Thai Restaurant,Hotel,Café,Smoke Shop
24,"North Midtown, The Annex, Yorkville",3,Café,Pizza Place,Coffee Shop,Italian Restaurant,Sandwich Place,Pub,Tea Room,History Museum,Mexican Restaurant,Vegetarian / Vegan Restaurant
26,"Chinatown, Grange Park, Kensington Market",3,Café,Bar,Chinese Restaurant,Vietnamese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Dessert Shop,Bakery,French Restaurant


#### Cluster 4 - Roselawn - "Chic" residential area

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,4,Playground,Pet Store,Music Venue,Home Service,Garden,Comic Shop,Diner,Farm,Falafel Restaurant,Event Space


#### Cluster 5 - "Chic" City area

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Riverdale, The Danforth West",5,Greek Restaurant,Coffee Shop,Pub,Café,Ice Cream Shop,Fast Food Restaurant,Italian Restaurant,Park,Yoga Studio,Discount Store
11,"Cabbagetown, St. James Town",5,Coffee Shop,Restaurant,Park,Café,Pharmacy,Pizza Place,Grocery Store,Pub,Thai Restaurant,Italian Restaurant
16,Berczy Park,5,Coffee Shop,Pizza Place,Hotel,Bistro,Molecular Gastronomy Restaurant,Liquor Store,Fish Market,Beach,Restaurant,Creperie
19,"Harbourfront East, Toronto Islands, Union Station",5,Coffee Shop,Aquarium,Hotel,Park,Brewery,Scenic Lookout,Plaza,Café,Baseball Stadium,Bubble Tea Shop
28,Stn A PO Boxes 25 The Esplanade,5,Cocktail Bar,Breakfast Spot,Beer Bar,Farmers Market,Seafood Restaurant,Bakery,Park,French Restaurant,Italian Restaurant,Tailor Shop
30,Christie,5,Korean Restaurant,Grocery Store,Coffee Shop,Café,Indian Restaurant,Diner,Park,Cocktail Bar,Karaoke Bar,Japanese Restaurant
31,"Dovercourt Village, Dufferin",5,Park,Coffee Shop,Camera Store,Bar,Supermarket,Café,Bakery,Portuguese Restaurant,Gym,Pharmacy
32,"Little Portugal, Trinity",5,Bar,Café,Coffee Shop,Restaurant,Bakery,Italian Restaurant,Asian Restaurant,Cocktail Bar,Tapas Restaurant,Theater
33,"Brockton, Exhibition Place, Parkdale Village",5,Coffee Shop,Bakery,Furniture / Home Store,Restaurant,Arts & Crafts Store,Bar,Café,Italian Restaurant,Music Venue,Lounge


#### Cluster 6 - Airport

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,"Bathurst Quay, CN Tower, Harbourfront West, Is...",6,Harbor / Marina,Airport Service,Boat or Ferry,Sculpture Garden,Coffee Shop,Airport Lounge,Boutique,Tunnel,Bar,Dog Run
